In [5]:
import json, csv, re
from collections import defaultdict
from pprint import pprint
from operator import itemgetter
import urllib.request
import urllib.parse
from scipy.stats import pearsonr

from IPython.core.debugger import Tracer; debug_here = Tracer()

In [25]:
# Article bias latest significance test
latestcsvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/articles_latest_citebias.csv'
cites = [] # citation diff/artlen
model = [] # revision bias score/artlen

with open(latestcsvpath) as latest:
    for row in csv.DictReader(latest):
        citeval = float(row['diff_norm'])
        modelval = float(row['bias_norm'])
        if (float(row['israeli_citations']) > 0 or float(row['palestinian_citations']) > 0) and \
                float(row['total_weight_words']) > 100:
            cites.append(citeval)
            model.append(modelval)

print(len(cites))
print(len(model))

984
984


In [26]:
pearsonr(cites, model)

(0.33171358751207464, 1.0565280237888783e-26)

In [23]:
# Article bias all revisions
csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/articles_citebias.csv'
cites = [] # citation diff/artlen
model = [] # revision bias score/artlen

with open(csvpath) as csvfile:
    for row in csv.DictReader(csvfile):
        citeval = float(row['diff_norm'])
        modelval = float(row['bias_norm'])
        if (float(row['israeli_citations']) > 0 or float(row['palestinian_citations']) > 0) and \
                float(row['total_weight_words']) > 100:
            cites.append(citeval)
            model.append(modelval)

print(len(cites))
print(len(model))

575341
575341


In [24]:
pearsonr(cites, model)

(0.31144533791424578, 0.0)

In [2]:
# Create csv for comparing article bias classifications (all articles)
artlen_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/article_lengths.csv'
pages = [line for line in csv.reader(open(artlen_csvpath,'r'))]
header = pages[0]
pages = pages[1:]
pagelens = {}
for line in pages:
    pagelens[tuple(line[:3])] = int(line[-1])

print("Done with lengths")

Done with lengths


In [29]:
# Create csv for comparing article bias classifications (just latest articles)
citations_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/citations_reverts.csv'
pages = [line for line in csv.reader(open(citations_csvpath,'r'))]
header = pages[0]
pages = pages[1:]
pagecites = {}
for line in pages:
    #pagecites[line[0]] = [float(el) for el in line[4:7]] # Gets latest revisions
    #pagecites[line[0]] = line # Gets latest revisions
    isrcite = int(line[4])
    palcite = int(line[5])
    diff = isrcite - palcite
    total = sum(int(el) for el in line[4:7])
    isrcite_norm = (0 if total==0 else float(isrcite)/total)
    palcite_norm = (0 if total==0 else float(palcite)/total)
    diff_norm = (0 if total==0 else float(diff)/total)
    pagecites[line[0]] = [el for el in line[:2]] + [float(el) for el in line[4:7]] + \
                            [diff, total, isrcite_norm, palcite_norm, diff_norm]

article_bias_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/ipc_article_biases.csv'
biases = [line for line in csv.reader(open(article_bias_csvpath,'r'))]
biasheader = biases[0]
biases = biases[1:]
pagebiases = {}
for b in biases:
    pagebiases[b[0]] = [float(el) for el in b[-6:-2]]
    
print(len(pagecites))
print(len(pagebiases))

csv_outpath = '/home/michael/school/cprose_research/wp/wikipedia/data/articles_latest_citebias.csv'
with open(csv_outpath, 'w') as out:
    writer = csv.writer(out)
    writer.writerow(header[:2] + header[4:7] + ['diff','total_cite','isrcite_norm','palcite_norm','diff_norm'] + \
                     biasheader[4:8])
    for art in sorted(pagecites.keys()):
        writer.writerow(pagecites[art] + pagebiases[art])
        #debug_here()
        
csv_outpath = '/home/michael/school/cprose_research/wp/wikipedia/data/articles_citebias.csv'
with open(csv_outpath, 'w') as out:
    writer = csv.writer(out)
    writer.writerow(header[:2] + header[4:7] + ['diff','total_cite','isrcite_norm','palcite_norm','diff_norm'] + \
                    ['art_length'] + biasheader[4:8] + [b+'_norm' for b in biasheader[4:8]])
    for art in sorted(pagecites.keys()):
        if art in pagebiases.keys():
            writer.writerow(pagecites[art] + [pagelens[art]] + list(pagebiases[art]))
        #debug_here()

2032
2032


In [39]:
#%debug
# Create csv for comparing article bias classifications (all articles)
artlen_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/article_lengths.csv'
pages = [line for line in csv.reader(open(artlen_csvpath,'r'))]
header = pages[0]
pages = pages[1:]
pagelens = {}
for line in pages:
    pagelens[tuple(line[:3])] = int(line[-1])

print("Done with lengths")

citations_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/citations_reverts.csv'
pages = [line for line in csv.reader(open(citations_csvpath,'r'))]
header = pages[0]
pages = pages[1:]
pagecites = {}
for line in pages:
    #pagecites[line[0]] = [float(el) for el in line[4:7]] # Gets latest revisions
    #pagecites[line[0]] = line # Gets latest revisions
    isrcite = int(line[4])
    palcite = int(line[5])
    diff = isrcite - palcite
    total = isrcite+palcite+int(line[6])
    artlen = pagelens[tuple(line[:3])]
    isrcite_norm = (0 if total==0 else float(isrcite)/artlen) # Doesn't make sense
    palcite_norm = (0 if total==0 else float(palcite)/artlen)
    diff_norm = (0 if total==0 else float(diff)/artlen)
    pagecites[tuple(line[:3])] = [el for el in line[:2]] + [float(el) for el in line[4:7]] + \
                            [diff, total, isrcite_norm, palcite_norm, diff_norm]

print("Done with citations")

article_bias_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/ipc_article_biases.csv'
biases = [line for line in csv.reader(open(article_bias_csvpath,'r'))]
biasheader = biases[0]
biases = biases[1:]
pagebiases = {}
for b in biases:
    artlen = pagelens[tuple(b[:3])]
    pagebiases[tuple(b[:3])] = [float(el) for el in b[-6:-2]] + [float(el)/artlen for el in b[-6:-2]]
    
print(len(pagecites))
print(len(pagebiases))

#%debug
csv_outpath = '/home/michael/school/cprose_research/wp/wikipedia/data/articles_citebias.csv'
with open(csv_outpath, 'w') as out:
    writer = csv.writer(out)
    writer.writerow(header[:2] + header[4:7] + ['diff','total_cite','isrcite_norm','palcite_norm','diff_norm'] + \
                    ['art_length'] + biasheader[4:8] + [b+'_norm' for b in biasheader[4:8]])
    for art in sorted(pagecites.keys()):
        if art in pagebiases.keys():
            writer.writerow(pagecites[art] + [pagelens[art]] + list(pagebiases[art]))
        #debug_here()

Done with lengths
Done with citations
1089176
1089171


In [3]:
# Editor csv
citationbias = dict(json.loads(open("/home/michael/school/cprose_research/wp/wikipedia/data/editor_citationbias.json",'r').read()))
len(citationbias)

204992

In [5]:
citationbias[list(citationbias.keys())[0]]

[0, 0.0]

In [23]:
def biases(csvpath):
    """
        Args:
            username: the user whose edits should be saved (for qualitative checking)
    """
    model_biases = defaultdict(float)
    numedits = defaultdict(int)
    
    with open(csvpath, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) # Skip first row
        prev_artname = ""
        prev_bias = 0
        for row in reader:
            art = row[0]
            username = row[2]
            numedits[username] += 1
            comment = row[3]
            bias = float(row[4])
            if not 'rvv' in comment and not 'vandal' in comment:
                if art == prev_artname:
                    bias_change = prev_bias - bias
                    model_biases[username] += bias_change
                else:
                    model_biases[username] += bias

            prev_artname = art
            prev_bias = bias
            
    return model_biases, numedits

In [36]:
def useredits(csvpath, selected):
    """
        Args:
            username: the user whose edits should be saved (for qualitative checking)
    """
    useredits = []
    
    with open(csvpath, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader) # Skip first row
        prev_artname = ""
        prev_bias = 0
        for row in reader:
            art = row[0]
            username = row[2]
            comment = row[3]
            bias = float(row[4])
            if not 'rvv' in comment and not 'vandal' in comment:
                if art == prev_artname:
                    bias_change = prev_bias - bias
                if username == selected:
                    useredits.append(row + [bias_change])

            prev_artname = art
            prev_bias = bias
            
    if useredits:
        useredits = sorted(useredits, key=itemgetter(-1))
    
    return useredits

In [26]:
# Build model biases dictionary
csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/ipc_article_biases.csv'

modelbiases, numedits = biases(csvpath)
            
print(len(list(modelbiases.keys())))
len(numedits)

204009


205362

In [32]:
# Get bot list
bots = []
cnum = ""
done = False

# Get bot category page
category_base_url = "https://en.wikipedia.org/w/api.php?" + \
    "action=query&" +  \
    "list=categorymembers&" + \
    "format=json&" + \
    "cmtitle=Category%3A{:s}&" +  \
    "cmprop=title&" + \
    "cmlimit=500" + \
    "&cmcontinue={:s}"
    
while not done:
    category_url = category_base_url.format(urllib.parse.quote("All_Wikipedia_bots"), cnum)
    #print(category_url)

    page =  urllib.request.urlopen(category_url).read().decode('utf-8')
    #print(page)

    # Extract bot names
    botsp = re.findall(r'"User\:(?P<username>[\w\ /\.]*)"\}', page)
    botsp = [re.sub(r'/.*', '', ed) for ed in botsp]
    #print(botsp[:10])
    bots += botsp
    print(len(bots))

    # Get continue number
    #cnum = re.search(r'cmcontinue":"(?P<cnum>[^"]*)"', page)
    cnum = re.search(r'cmcontinue":"([^"]*)"', page)
    if cnum:
        cnum = cnum.group(1)
        #print(cnum)
    else:
        done = True

# Add other bots
other_bots = ['SmackBot', 'Tawkerbot2']
bots += other_bots
len(bots)

441
872
1308
1477


1479

In [ ]:
# Get revision lengths
artlen_csvpath = '/home/michael/school/cprose_research/wp/wikipedia/data/article_lengths.csv'
pages = [line for line in csv.reader(open(artlen_csvpath,'r'))]
header = pages[0]
pages = pages[1:]
pagelens = {}
for line in pages:
    pagelens[tuple(line[:3])] = int(line[-1])

print("Done with lengths")

In [42]:
pagelens[('Shabbat', '2008-12-24 16:21:45', 'Glane23')]

3612

In [41]:
pagelens[('Tzipi Livni', '2009-12-21 16:18:53', '82.1.54.219')]

3242

In [33]:
# Check sum_bias top editors
sorted_modelbias = sorted([item for item in modelbiases.items() 
                           if not re.match(r'^[?\ �]+$',item[0]) and numedits[item[0]] > 5 and \
                              item[0] not in bots], key=itemgetter(1))
pprint(sorted_modelbias[:10])
pprint(sorted_modelbias[-10:])

[('Peter Deer', -25343.121521899342),
 ('Jayjg', -18635.96141006936),
 ('ResearchFinder', -18206.084239807526),
 ('Tawkerbot4', -11948.824870282524),
 ('Curps', -11132.559114150774),
 ('Cureden', -10992.294565723589),
 ('Wiki alf', -9344.224974998313),
 ('82.1.54.219', -8569.240751205996),
 ('Cometstyles', -7980.491574382942),
 ('El C', -6935.012873889626)]
[('Jon513', 7983.616634151831),
 ('Glane23', 8289.253614389705),
 ('Antonio Lopez', 10728.471832793477),
 ('Ixfd64', 14495.444826636338),
 ('212.98.136.42', 16586.595097739915),
 ('Pan Dan', 18217.451190746408),
 ('HistoryBuffEr', 21005.42305323439),
 ('Sjakkalle', 35590.582086142625),
 ('Mike Rosoft', 197060.75086122236),
 ('Yamamoto Ichiro', 198915.87273713664)]


In [44]:
#%debug
# Get most distinguishing edits for a certain editor
edits = useredits(csvpath, 'Ixfd64')
#edits[:3]
edits[-3:]

[['Jews',
  '2005-11-03 03:09:25',
  'Ixfd64',
  'Reverted edits by [[Special:Contributions/193.62.43.210|193.62.43.210]] to last version by Ixfd64',
  '293.58786455966424',
  '3147.2750787444716',
  '-952.70814877236853',
  '1486.57807581262',
  '1',
  '1',
  7.5735113701366],
 ['Jews',
  '2005-09-23 21:29:37',
  'Ixfd64',
  'Reverted edits by [[Special:Contributions/Jtkiefer|Jtkiefer]] to last version by 203.221.152.225',
  '-0.06750087992651628',
  '2.9999999999999991',
  '-0.76861850051644254',
  '1.8999747228493362',
  '1',
  '1',
  285.09827532518244],
 ['Jews',
  '2006-06-30 15:39:13',
  'Ixfd64',
  'Reverted edits by [[Special:Contributions/66.61.54.253|66.61.54.253]] ([[User talk:66.61.54.253|talk]]) to last version by Njaard',
  '332.6212056077128',
  '3713.4267031472164',
  '-1147.0089153824742',
  '1756.9420277445647',
  '1',
  '1',
  14958.139856889391]]

In [ ]:
editorbiases = {}
    if b[2] not in editorbiases:
    editorbiases[b[2]] = []
    if b[0] in pagebiases:
    editorbiases[b[2]].append([float(el)-prv for (el,prv) in zip(b[-6:-2],pagebiases[b[0]])])